In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
(sqrt(3)*I - sqrt(5)).minpoly()

x^4 - 4*x^2 + 64

In [3]:
alg_5 = 1/16*(sqrt(3)*I - sqrt(5))^3 - 3/4*(sqrt(3)*I - sqrt(5))

In [4]:
alg_5.expand()

sqrt(5)

In [5]:
# sqrt(5) = 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5

In [6]:
alg_i = 1/16*(sqrt(3)*I - sqrt(5))^3 + 1/4*(sqrt(3)*I - sqrt(5))

In [7]:
alg_i.expand()

I*sqrt(3)

In [8]:
# i sqrt(3) =  1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5

In [9]:
xx = QQ['xx'].0

In [10]:
Q.<i_rad3_minus_rad5> = NumberField(xx^4 - 4*xx^2 + 64)

In [11]:
i_rad3 = 1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5
rad5 = 1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5

In [12]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [13]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})
SE2 = SE1.subs({d:(c*c+e*f)/c})
SE3 = SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)})
SE4 = SE3.subs({f:c*(3*c-e)/(c+e)})
SE6 = SE3.subs({f: -c*(5*c+e)/(c+e)})
SE9 = SE3.subs({e : i_rad3*c})
SE10 = SE4.subs({e : (2-rad5)*c})
SE18 = SE4.subs({e : i_rad3*c})

In [14]:
# STUDY SUBFAMILIES

In [15]:
def get_unique_conditions(cubic):
    ideals = []
    gb = []
    table = []
    for pl in cubic.tritangent_planes:
        if pl.labels in [plane.labels for plane in cubic.tritangent_planes if plane.conditions==0]:
            continue
        poly = pl.conditions
        flag = True
        for ideal in ideals:
            if poly in ideal:
                table.append([pl.labels, gb.index(ideal.groebner_basis()[0])+1])
                flag = False
                break
        if flag == True:
            ideals.append(P.ideal(poly))
            gb.append(P.ideal(poly).groebner_basis()[0])
            table.append([pl.labels, gb.index(P.ideal(poly).groebner_basis()[0])+1])
    return [poly*poly.denominator() for poly in gb], table

In [16]:
# CONDITIONS FOR SE3

In [17]:
Q3, table3 = get_unique_conditions(SE3)

In [18]:
for i in range(1,len(Q3)+1):
    print(Q3[i-1], [el[0] for el in table3 if el[1]==i])

5*c^2 - c*e - c*f + e*f [['E1', 'G2', 'F12'], ['E3', 'G1', 'F13'], ['E4', 'G3', 'F34']]
3*c^2 + c*e + c*f - e*f [['E1', 'G3', 'F13']]
c^2 + 3*c*e - c*f + e*f [['E1', 'G5', 'F15'], ['E6', 'G3', 'F36'], ['F13', 'F25', 'F46']]
c^2 - c*e + 3*c*f + e*f [['E1', 'G6', 'F16'], ['E5', 'G3', 'F35'], ['F13', 'F26', 'F45']]
5*c^2 + c*e + c*f + e*f [['E2', 'G1', 'F12'], ['E3', 'G4', 'F34'], ['E4', 'G2', 'F24']]
3*c^2 - c*e - c*f - e*f [['E2', 'G4', 'F24']]
c^2 + c*e - 3*c*f + e*f [['E2', 'G5', 'F25'], ['E6', 'G4', 'F46'], ['F15', 'F24', 'F36']]
c^2 - 3*c*e + c*f + e*f [['E2', 'G6', 'F26'], ['E5', 'G4', 'F45'], ['F16', 'F24', 'F35']]
c^2 + e*f [['E3', 'G2', 'F23'], ['E4', 'G1', 'F14'], ['F12', 'F34', 'F56']]
3*c^2 + e^2 [['E3', 'G5', 'F35'], ['E4', 'G6', 'F46'], ['E5', 'G2', 'F25'], ['E6', 'G1', 'F16'], ['F12', 'F36', 'F45'], ['F15', 'F26', 'F34']]
3*c^2 + f^2 [['E3', 'G6', 'F36'], ['E4', 'G5', 'F45'], ['E5', 'G1', 'F15'], ['E6', 'G2', 'F26'], ['F12', 'F35', 'F46'], ['F16', 'F25', 'F34']]
2*c - e + 

In [19]:
[Q3[1], Q3[5], Q3[13]]

[3*c^2 + c*e + c*f - e*f, 3*c^2 - c*e - c*f - e*f, e + f]

In [20]:
# SUBFAMILIES OF SE3 WITH 4 ECKARDT POINTS

In [21]:
k3 = SE3.subs({f:-e})
k3.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['F13', 'F24', 'F56'],
 ['F14', 'F23', 'F56']]

In [22]:
k1 = SE3.subs({f:c*(3*c+e)/(e-c)})
k1.eckardt_points_labels

[['E1', 'G3', 'F13'],
 ['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['F13', 'F24', 'F56']]

In [23]:
SE4.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['E2', 'G4', 'F24'],
 ['F13', 'F24', 'F56']]

In [24]:
L_set_k3 = ('E1', 'F14', 'F23', 'G3', 'G1')
L_set_k1 = ('G4', 'E1', 'G3', 'E2', 'G2')
L_set_SE4 = ('G3', 'E2', 'G4', 'E1', 'G1')

In [25]:
proj_k3_SE4 = find_projectivity(SE4.get_L_set_in_plucker(L_set_SE4), k3.get_L_set_in_plucker(L_set_k3))
proj_k1_SE4 = find_projectivity(SE4.get_L_set_in_plucker(L_set_SE4), k1.get_L_set_in_plucker(L_set_k1))

In [26]:
proj_k3_SE4_inv = proj_k3_SE4.inverse()
proj_k3_SE4_inv = (proj_k3_SE4_inv*proj_k3_SE4_inv.denominator()).change_ring(P)
change_coordinates_proj_k3_SE4_inv = change_coord(proj_k3_SE4_inv)
proj_k1_SE4_inv = proj_k1_SE4.inverse()
proj_k1_SE4_inv = (proj_k1_SE4_inv*proj_k1_SE4_inv.denominator()).change_ring(P)
change_coordinates_proj_k1_SE4_inv = change_coord(proj_k1_SE4_inv)

In [27]:
k3_inverse_SE4 = SE4.subs(change_coordinates_proj_k3_SE4_inv)
k1_inverse_SE4 = SE4.subs(change_coordinates_proj_k1_SE4_inv)

In [28]:
mon=((x+y+z+t)^3).monomials()

In [29]:
print([k3_inverse_SE4.coefficient(mn) for mn in mon])

[0, c^5 - 1/3*c^4*e - 1/9*c^3*e^2 - 1/9*c^2*e^3, -1/4*c^5 + 1/12*c^4*e + 1/9*c^3*e^2 + 1/9*c^2*e^3 - 1/12*c*e^4 + 1/36*e^5, 0, -2/9*c^5 + 2/9*c^3*e^2, 7/18*c^5 - 1/6*c^4*e - 1/6*c^3*e^2 - 1/18*c^2*e^3, 0, -1/9*c^5 + 1/9*c^3*e^2, 0, 0, 0, -11/9*c^5 + 1/3*c^4*e - 1/9*c^3*e^2 + 1/9*c^2*e^3, 5/36*c^5 - 1/12*c^4*e - 1/9*c^2*e^3 + 1/12*c*e^4 - 1/36*e^5, 0, -1/6*c^5 + 1/6*c^4*e - 1/18*c^3*e^2 + 1/18*c^2*e^3, 0, 0, 4/9*c^5, 0, 0]


In [30]:
print([k3.eqn.coefficient(mn) for mn in mon])

[0, 2*e^2, -2*e^2, 0, c^2 - e^2, 1/2*c^2 + 3/2*e^2, 0, 1/2*c^2 - 1/2*e^2, 0, 0, 0, c^2 - e^2, 1/2*c^2 + 3/2*e^2, 0, -3/2*c^2 - 1/2*e^2, 0, 0, -2*c^2, 0, 0]


In [31]:
# SUBFAMILIES OF SE3 WITH 6 ECKARDT POINTS

In [32]:
# SUBFAMILIES OF SE3 WITH 9 ECKARDT POINTS

In [33]:
SE9_1 = SE3.subs({e : -i_rad3*c})

In [34]:
L_set_SE9 = ['F46', 'G6', 'F26', 'F15', 'E3']
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [35]:
proj_SE9 = find_projectivity(SE9.get_L_set_in_plucker(L_set_SE9), SE9.get_L_set_in_plucker(L_set_base))
proj_SE9 = proj_SE9/(proj_SE9.list()[0].coefficients()[0])
proj_SE9 = proj_SE9*proj_SE9.denominator()

In [36]:
proj_SE9_inv = proj_SE9.inverse()
proj_SE9_inv = (proj_SE9_inv*proj_SE9_inv.denominator()).change_ring(P)
change_coordinates_SE9_inv = change_coord(proj_SE9_inv)

In [37]:
new_eqn_SE9_1 = SE9_1.subs({c:f-c, f: -c+2*i_rad3*c-f})
inverse_SE9 = SE9.subs(change_coordinates_SE9_inv)

In [38]:
are_cubic_same(new_eqn_SE9_1, inverse_SE9)

True

In [39]:
# CONDITIONS FOR SE4

In [40]:
Q4, table4 = get_unique_conditions(SE4)

In [41]:
for i in range(1,len(Q4)+1):
    print(Q4[i-1].factor(), [el[0] for el in table4 if el[1]==i])
    print()

(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 + 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 + 2)*e) [['E1', 'G2', 'F12'], ['E3', 'G1', 'F13'], ['E3', 'G2', 'F23'], ['E4', 'G1', 'F14'], ['E4', 'G3', 'F34'], ['F12', 'F34', 'F56']]

(3) * (c + (1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*e) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*e) [['E1', 'G3', 'F13'], ['E3', 'G5', 'F35'], ['E3', 'G6', 'F36'], ['E4', 'G5', 'F45'], ['E4', 'G6', 'F46'], ['E5', 'G1', 'F15'], ['E5', 'G2', 'F25'], ['E6', 'G1', 'F16'], ['E6', 'G2', 'F26'], ['F12', 'F35', 'F46'], ['F12', 'F36', 'F45'], ['F14', 'F23', 'F56'], ['F15', 'F26', 'F34'], ['F16', 'F25', 'F34']]

(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*e) [['E1', 'G5', 'F15'], ['E6', 'G3', 'F36'], ['E6', 'G5', 'F56'], ['F13', 'F25', 'F46'], ['F14', 'F25', 'F36'], ['F15', 'F23', 'F46']]

(5) * (c + (1/80*i_rad3_minus_rad5^3 - 3/20

In [42]:
# SUBFAMILIES OF SE4 WITH 10 ECKARDT POINTS

In [43]:
# CONDITIONS FOR SE6

In [44]:
Q6, table6 = get_unique_conditions(SE6)

In [45]:
for i in range(1,len(Q6)+1):
    print(Q6[i-1].factor(), [el[0] for el in table6 if el[1]==i])

(5) * (c + (1/80*i_rad3_minus_rad5^3 - 3/20*i_rad3_minus_rad5)*e) * (c + (-1/80*i_rad3_minus_rad5^3 + 3/20*i_rad3_minus_rad5)*e) [['E1', 'G2', 'F12'], ['E3', 'G1', 'F13'], ['E4', 'G3', 'F34'], ['F14', 'F23', 'F56']]
(c + (1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5 - 2)*e) * (c + (-1/16*i_rad3_minus_rad5^3 + 3/4*i_rad3_minus_rad5 - 2)*e) [['E1', 'G3', 'F13'], ['E3', 'G2', 'F23'], ['E4', 'G1', 'F14'], ['F12', 'F34', 'F56']]
(3) * (c + (1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*e) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*e) [['E1', 'G5', 'F15'], ['E3', 'G5', 'F35'], ['E4', 'G6', 'F46'], ['E5', 'G2', 'F25'], ['E5', 'G6', 'F56'], ['E6', 'G1', 'F16'], ['E6', 'G3', 'F36'], ['F12', 'F36', 'F45'], ['F13', 'F25', 'F46'], ['F14', 'F26', 'F35'], ['F15', 'F26', 'F34'], ['F16', 'F23', 'F45']]
(7) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 + 2/7)*e) * (c + (-1/112*i_rad3_minus_rad5^3 - 1/28*i_rad3_minus_rad5 + 2/7)*e) [['E1', 'G6', 'F16'], ['E3', 'G6'

In [46]:
# SUBFAMILIES OF SE6 WITH 10 ECKARDT POINTS

In [47]:
# SUBFAMILIES OF SE6 WITH 18 ECKARDT POINTS

In [48]:
# CONDITIONS FOR SE9

In [49]:
Q9, table9 = get_unique_conditions(SE9)

In [50]:
for i in range(1,len(Q9)+1):
    print(Q9[i-1].factor(), [el[0] for el in table9 if el[1]==i])

(112) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 - 2/7)*f) [['E1', 'G2', 'F12'], ['E2', 'G6', 'F26'], ['E3', 'G1', 'F13'], ['E4', 'G3', 'F34'], ['E5', 'G4', 'F45'], ['E6', 'G5', 'F56'], ['F14', 'F25', 'F36'], ['F15', 'F23', 'F46'], ['F16', 'F24', 'F35']]
(48) * (c + (-1/48*i_rad3_minus_rad5^3 - 1/12*i_rad3_minus_rad5)*f) [['E1', 'G3', 'F13'], ['E2', 'G4', 'F24'], ['E3', 'G6', 'F36'], ['E4', 'G5', 'F45'], ['E5', 'G1', 'F15'], ['E6', 'G2', 'F26'], ['F12', 'F35', 'F46'], ['F14', 'F23', 'F56'], ['F16', 'F25', 'F34']]
(112) * (c + (1/112*i_rad3_minus_rad5^3 + 1/28*i_rad3_minus_rad5 + 2/7)*f) [['E1', 'G5', 'F15'], ['E2', 'G1', 'F12'], ['E3', 'G4', 'F34'], ['E4', 'G2', 'F24'], ['E5', 'G6', 'F56'], ['E6', 'G3', 'F36'], ['F13', 'F25', 'F46'], ['F14', 'F26', 'F35'], ['F16', 'F23', 'F45']]
(16) * (c + (1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5)*f) [['E1', 'G6', 'F16'], ['E2', 'G5', 'F25'], ['E3', 'G2', 'F23'], ['E4', 'G1', 'F14'], ['E5', 'G3', 'F35'], ['E6', 'G4', 'F46'], [

In [51]:
[solve_linear_system_in_fraction_field([poly], [f], [c]) for poly in Q9]

[[(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5 + 2)*c],
 [(-1/16*i_rad3_minus_rad5^3 - 1/4*i_rad3_minus_rad5)*c],
 [(1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5 - 2)*c],
 [(1/48*i_rad3_minus_rad5^3 + 1/12*i_rad3_minus_rad5)*c]]

In [52]:
# SUBFAMILIES OF SE9 WITH 18 ECKARDT POINTS

In [53]:
i_rad3

1/16*i_rad3_minus_rad5^3 + 1/4*i_rad3_minus_rad5

In [54]:
rad5

1/16*i_rad3_minus_rad5^3 - 3/4*i_rad3_minus_rad5

In [55]:
# Lemma 2.2.5

In [56]:
eqn = SE10.eqn